In [1]:
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt


In [2]:

def get_ping_pong(trace):
    orgs = trace["org_group"].tolist()
    timestamps = trace["timestamp"].tolist()
    orgs = [(x, timestamps[i]) for i, x in enumerate(orgs) if i ==0 or x != orgs[i-1] ]
    pairs = []
    n = len(orgs)
    for ping_idx in range(n):
        ping_item = orgs[ping_idx][0]
        ping_timestamp = orgs[ping_idx][1]
        if ping_item in [x[0] for x in orgs[ping_idx:]]:
            for pong_idx in range(ping_idx+1, n):
                ping_again = orgs[pong_idx][0]
                if ping_item == ping_again:
                    duration_days = (orgs[pong_idx][1] - ping_timestamp).total_seconds() / (24*60*60)
                    pairs.append(((ping_item, orgs[pong_idx - 1][0]), (duration_days)))      
                    break
    return pairs

def ping_pong_trace_serie(trace):
    pairs = get_ping_pong(trace)
    return pd.Series({
        "case_seq_num": trace.name,
        "has_ping_pong": len(pairs) > 0,
        "ping_pong_pairs": pairs
    })


In [12]:

df = pd.read_pickle("./data/1_log_clean.pkl")
ping_pong_df = df.groupby("case_seq_num").apply(ping_pong_trace_serie).reset_index(drop=True)
all_durations = [pair[1] for pairs in ping_pong_df["ping_pong_pairs"] for pair in pairs]




C:\Users\antoi\AppData\Local\Temp\ipykernel_29476\3621845974.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ping_pong_df = df.groupby("case_seq_num").apply(ping_pong_trace_serie).reset_index(drop=True)


In [15]:
df["lifecycle_transition"].unique()

array(['Awaiting Assignment', 'In Progress', 'Wait - User', 'Resolved',
       'Closed', 'Wait', 'Assigned', 'In Call'], dtype=object)

In [11]:
df = df[df["case_seq_num"] == "1-647788578"]
df

,org_group,res_country,org_country,org_res,org_involved,org_role,concept_name,impact,product,lifecycle_transition,timestamp,case_seq_num
1670,D5,Belgium,be,Katia,Org line C,A2_1,Accepted,Medium,PROD542,In Progress,2012-01-19 10:36:47+00:00,1-647788578
1671,D5,Belgium,be,Katia,Org line C,A2_1,Accepted,Medium,PROD542,In Progress,2012-01-19 10:37:28+00:00,1-647788578
1672,D5,Belgium,be,Katia,Org line C,A2_1,Accepted,Medium,PROD542,In Progress,2012-01-19 10:45:57+00:00,1-647788578
1673,D4,Belgium,be,Katia,Org line A2,A2_1,Queued,Medium,PROD542,Awaiting Assignment,2012-01-19 10:46:08+00:00,1-647788578
1674,D5,Belgium,be,Katia,Org line C,A2_1,Accepted,Medium,PROD542,In Progress,2012-01-19 10:46:38+00:00,1-647788578
1675,V37 2nd,Belgium,be,Katia,Org line V7n,NaN,Queued,Medium,PROD542,Awaiting Assignment,2012-01-19 10:47:36+00:00,1-647788578
1676,V37 2nd,Netherlands,be,Danny,Org line V7n,NaN,Accepted,Medium,PROD542,In Progress,2012-01-19 15:43:58+00:00,1-647788578
1677,V32 2nd,Netherlands,be,Danny,Org line V7n,NaN,Queued,Medium,PROD542,Awaiting Assignment,2012-01-19 16:41:15+00:00,1-647788578
1678,V32 2nd,Netherlands,be,Earl,Org line V7n,NaN,Accepted,Medium,PROD542,In Progress,2012-03-27 15:58:10+00:00,1-647788578
1679,V37 2nd,Netherlands,be,Juan,Org line V7n,NaN,Accepted,Medium,PROD542,In Progress,2012-05-04 15:06:27+00:00,1-647788578


In [6]:
dd = pd.DataFrame(all_durations, columns=["duration"])
dd.describe()



,duration
count,3026.000000
mean,5.376455
std,11.406737
min,0.000243
25%,0.156881
50%,1.188814
75%,5.833333
max,110.123669


In [5]:
ping_pong_df.head(2)

,case_seq_num,has_ping_pong,ping_pong_pairs
0,1-642402641,False,[]
1,1-642714990,True,"[((D6, V37 2nd), 6.938310185185185), ((V37 2nd..."


In [8]:

pair_df = pd.DataFrame(
    [
        (row.case_seq_num, p, d)
        for _, row in ping_pong_df.iterrows()
        for (p, d) in row.ping_pong_pairs
    ],
    columns=["case_seq_num", "pair", "duration"]
)
idx_max = pair_df.groupby("pair")["duration"].idxmax()
max_cases = (
    pair_df
    .loc[idx_max, ["case_seq_num", "pair", "duration"]]
    .sort_values("duration", ascending=False)
    .head(10)                       # top 10 pairs, change to .head(5) for top 5
    .reset_index(drop=True)
)

print("Top 5 ping pong pairs by max duration (with case):")
for _, row in max_cases.head(5).iterrows():
    a, b = row["pair"]
    print(f"Case {row['case_seq_num']}: {a} <-> {b} : {row['duration']:.2f} days")


Top 5 ping pong pairs by max duration (with case):
Case 1-647788578: D5 <-> V37 2nd : 110.12 days
Case 1-642714990: V37 2nd <-> V32 2nd : 109.03 days
Case 1-651196130: D6 <-> V46 2nd : 105.88 days
Case 1-655281402: G18 3rd <-> L23 3rd : 105.10 days
Case 1-652256138: D2 <-> V51 2nd : 103.88 days


In [38]:
idx_max = pair_df.groupby("pair")["duration"].idxmax()
max_cases = (
    pair_df
    .loc[idx_max, ["case_seq_num", "pair", "duration"]]
    .sort_values("duration", ascending=False)
    .head(10)                       # top 10 pairs, change to .head(5) for top 5
    .reset_index(drop=True)
)

print("Top 5 ping pong pairs by max duration (with case):")
for _, row in max_cases.head(5).iterrows():
    a, b = row["pair"]
    print(f"Case {row['case_seq_num']}: {a} <-> {b} : {row['duration']:.2f} days")


Top 5 ping pong pairs by max duration (with case):
Case 1-647788578: D5 <-> V37 2nd : 110.12 days
Case 1-642714990: V37 2nd <-> V32 2nd : 109.03 days
Case 1-651196130: D6 <-> V46 2nd : 105.88 days
Case 1-655281402: G18 3rd <-> L23 3rd : 105.10 days
Case 1-652256138: D2 <-> V51 2nd : 103.88 days
